In [1]:
%load_ext autoreload
%autoreload 2
from autoseg.datasets import get_dataset_path
from autoseg.config import read_config
from autoseg.datasets.utils import get_shape, get_voxel_size
import networkx as nx
import zarr
import numpy as np
from funlib.evaluate import (
    rand_voi,
    expected_run_length,
    get_skeleton_lengths,
)
from functools import reduce
import pandas as pd

/home/anton/miniforge3/envs/autoseg2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [200]:
arr = np.load("/home/anton/github/autoseg/src/autoseg/artifacts/UNet_OSA_run_1/predictions/step-200000/oblique_prediction.zarr/luts/fragment_segment/seg_edges_mean_50.npz")

In [206]:
pd.DataFrame(arr["fragment_segment_lut"].transpose())

,0,1
0,11901178,9
1,11901179,1
2,11901180,9
3,11901181,9
4,11901182,9
...,...,...
43293,324310213,41276
43294,324310214,43294
43295,324310215,33719
43296,324310216,41276


In [111]:
product = lambda it: reduce(lambda x, y: x * y, it)

In [195]:
def get_dataset_stats(dataset):
  z = get_dataset_path(dataset + ".zarr")
  s = get_dataset_path(dataset + ".graphml")
  skeletons = nx.read_graphml(s)


  number_of_edges = skeletons.number_of_edges()
  number_of_nodes = skeletons.number_of_nodes()
  number_of_objects = nx.number_connected_components(skeletons)

  skeleton_lengths = get_skeleton_lengths(
    skeletons,
    skeleton_position_attributes=["position_z", "position_y", "position_x"],
    skeleton_id_attribute="id",
    store_edge_length="length"
  )

  total_skeleton_length = np.sum([l for _, l in skeleton_lengths.items()])
  average_skeleton_length = total_skeleton_length / number_of_objects
  std_skeleton_length = np.std([l for _, l in skeleton_lengths.items()])
  shortest_skeleton_length = min([l for _, l in skeleton_lengths.items()])
  longest_skeleton_length = max([l for _, l in skeleton_lengths.items()])

  voxel_size = get_voxel_size(z, "volumes/image/s1")

  shape_vx = get_shape(z, "volumes/image/s1")
  volume_vx = product(shape_vx)

  to_nm = lambda shape: [v*s for v, s in zip(list(voxel_size), list(shape))]

  size_nm = to_nm(shape_vx) 
  volume_nm = product(size_nm)

  image = zarr.open(z)
  labelled_vx = np.count_nonzero(image["volumes"]["neuron_ids"]["s1"])
  labelled_nm = labelled_vx * product(voxel_size)

  mask = np.array(image["volumes"]["object_mask"]["s1"])
  object_indices = np.argwhere(mask > 0)
  min_indices = object_indices.min(axis=0)
  max_indices = object_indices.max(axis=0)
  volume_used_vx = product(max_indices - min_indices)
  volume_used_nm = volume_used_vx * product(voxel_size)

  percent_volume_used = (volume_used_vx / volume_vx) * 100
  percent_volume_labelled_of_used = (labelled_vx / volume_used_vx) * 100
  return {
    "Voxel Size": "(" + "nm, ".join(map(str, voxel_size)) + "nm)",
    "Total Volume (nm^3)": volume_nm,
    "Labelled Volume (nm^3)": labelled_nm,
    "Used Volume (nm^3)": volume_used_nm,
    "Percent of Used Volume Labelled": str(round(percent_volume_labelled_of_used, 1)) + "%",
    "Number of Objects": number_of_objects,
    "Number of Nodes in Skeleton": number_of_nodes,
    "Number of Edges in Skeleton": number_of_edges,
    "Sum of Skeleton Lengths (nm)": round(total_skeleton_length),
    "Average Skeleton Length (nm)": round(average_skeleton_length),
    "Standard Deviation of Skeleton Length (nm)": round(std_skeleton_length),
    "Shortest Skeleton Length (nm)": round(shortest_skeleton_length),
    "Longest Skeleton Length (nm)": round(longest_skeleton_length)
  }


In [196]:
datasets = ['BBCHZ', 'BLSNK', 'BRNLG', 'CLZBJ', 'CRQCR', 'CSKBZ', 'DRZNC', 'DTZVX', 'GFBZX', 'HNVRR', 'HVCBQ', 'JJPSC', 'KSGRS', 'MCZJJ', 'MFBCF', 'MPLTJ', 'NDKZB', 'QRFNB', 'RFHTC', 'RJZQR', 'RLCVK', 'SRQHN', 'TYLYL', 'YSJNL']
dataset_stats = [get_dataset_stats("SynapseWeb/team_dentate/" + ds) for ds in datasets]
for stat, ds in zip(dataset_stats, datasets):
  stat["Dataset"] = ds
df = pd.DataFrame.from_dict(dataset_stats)
df.set_index("Dataset", inplace=True)

ds = dataset_stats
print("Average Dataset Volume", sum([d["Used Volume (nm^3)"] for d in ds])/len(ds))
print("Average Labelled Volume", sum([d["Labelled Volume (nm^3)"] for d in ds])/len(ds))
print("Percent of Volume Labelled", sum([d["Labelled Volume (nm^3)"] for d in ds])/sum([d["Used Volume (nm^3)"] for d in ds]))
print("Average Number of Objects", sum([d["Number of Objects"] for d in ds])/len(ds))
print("Average Skeleton Length", sum([d["Average Skeleton Length (nm)"] for d in ds])/len(ds))
print(df.to_latex().replace("%", "\%").replace("nm^3", "$nm^3$"))
df

/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/BBCHZ.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/BLSNK.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/BRNLG.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/CLZBJ.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/CRQCR.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/CSKBZ.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/DRZNC.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/DTZVX.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/GFBZX.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/HNVRR.zarr volumes/image/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/HVCBQ

,Voxel Size,Total Volume (nm^3),Labelled Volume (nm^3),Used Volume (nm^3),Percent of Used Volume Labelled,Number of Objects,Number of Nodes in Skeleton,Number of Edges in Skeleton,Sum of Skeleton Lengths (nm),Average Skeleton Length (nm),Standard Deviation of Skeleton Length (nm),Shortest Skeleton Length (nm),Longest Skeleton Length (nm)
Dataset,,,,,,,,,,,,,
BBCHZ,"(50nm, 4nm, 4nm)",510160320000,30469896000,327407376800,9.3%,90,57232,57142,412746,4586,5456,152,24156
BLSNK,"(50nm, 4nm, 4nm)",985695480000,8377858400,419114640000,2.0%,208,31468,31260,203701,979,2790,51,35727
BRNLG,"(50nm, 4nm, 4nm)",401593040000,12064209600,213015308800,5.7%,13,8999,8986,87977,6767,8137,153,19674
CLZBJ,"(50nm, 4nm, 4nm)",385377400000,6284678400,267524280800,2.3%,40,15563,15523,121245,3031,7485,37,36828
CRQCR,"(50nm, 4nm, 4nm)",3148495360000,10823156000,682344992000,1.6%,52,18187,18135,140705,2706,7798,48,48253
CSKBZ,"(50nm, 4nm, 4nm)",418701920000,3742130400,111946380000,3.3%,13,5910,5897,54700,4208,5557,123,16165
DRZNC,"(50nm, 4nm, 4nm)",1140951840000,10198180800,428892688000,2.4%,88,20961,20873,166695,1894,5150,56,31283
DTZVX,"(50nm, 4nm, 4nm)",652695680000,8833390400,207951139200,4.2%,18,11889,11871,99089,5505,7043,48,17975
GFBZX,"(50nm, 4nm, 4nm)",3125470400000,7561151200,1384928160000,0.5%,41,12881,12840,96543,2355,6301,69,26406


In [197]:
def get_dataset_stats_kh2015(dataset):
  z = get_dataset_path(dataset)
  s = get_dataset_path(dataset + ".graphml")
  s = str(s).replace(".zarr.zip", "")
  skeletons = nx.read_graphml(s)


  number_of_edges = skeletons.number_of_edges()
  number_of_nodes = skeletons.number_of_nodes()
  number_of_objects = nx.number_connected_components(skeletons)

  skeleton_lengths = get_skeleton_lengths(
    skeletons,
    skeleton_position_attributes=["position_z", "position_y", "position_x"],
    skeleton_id_attribute="id",
    store_edge_length="length"
  )

  total_skeleton_length = np.sum([l for _, l in skeleton_lengths.items()])
  average_skeleton_length = total_skeleton_length / number_of_objects
  std_skeleton_length = np.std([l for _, l in skeleton_lengths.items()])
  shortest_skeleton_length = min([l for _, l in skeleton_lengths.items()])
  longest_skeleton_length = max([l for _, l in skeleton_lengths.items()])

  voxel_size = get_voxel_size(z, "raw/s1")

  shape_vx = get_shape(z, "labels/s1")
  volume_vx = product(shape_vx)

  to_nm = lambda shape: [v*s for v, s in zip(list(voxel_size), list(shape))]

  size_nm = to_nm(shape_vx) 
  volume_nm = product(size_nm)

  image = zarr.open(z)
  labelled_vx = np.count_nonzero(image["labels"]["s1"])
  labelled_nm = labelled_vx * product(voxel_size)

  mask = np.array(image["labels_mask"]["s1"])
  object_indices = np.argwhere(mask > 0)
  min_indices = object_indices.min(axis=0)
  max_indices = object_indices.max(axis=0)
  volume_used_vx = product(max_indices - min_indices)
  volume_used_nm = volume_used_vx * product(voxel_size)

  percent_volume_used = (volume_used_vx / volume_vx) * 100
  percent_volume_labelled_of_used = (labelled_vx / volume_used_vx) * 100
  return {
    "Voxel Size": "(" + "nm, ".join(map(str, voxel_size)) + "nm)",
    "Total Volume (nm^3)": volume_nm,
    "Labelled Volume (nm^3)": labelled_nm,
    "Used Volume (nm^3)": volume_used_nm,
    "Percent of Used Volume Labelled": str(round(percent_volume_labelled_of_used, 1)) + "%",
    "Number of Objects": number_of_objects,
    "Number of Nodes in Skeleton": number_of_nodes,
    "Number of Edges in Skeleton": number_of_edges,
    "Sum of Skeleton Lengths (nm)": round(total_skeleton_length),
    "Average Skeleton Length (nm)": round(average_skeleton_length),
    "Standard Deviation of Skeleton Length (nm)": round(std_skeleton_length),
    "Shortest Skeleton Length (nm)": round(shortest_skeleton_length),
    "Longest Skeleton Length (nm)": round(longest_skeleton_length)
  }

In [198]:
datasets = ["oblique", "apical", "spine"]
dataset_stats = [get_dataset_stats_kh2015("SynapseWeb/kh2015/" + ds) for ds in datasets]
for stat, ds in zip(dataset_stats, datasets):
  stat["Dataset"] = ds
df = pd.DataFrame.from_dict(dataset_stats)
df.set_index("Dataset", inplace=True)

ds = dataset_stats
print("Average Dataset Volume", sum([d["Used Volume (nm^3)"] for d in ds])/len(ds))
print("Average Labelled Volume", sum([d["Labelled Volume (nm^3)"] for d in ds])/len(ds))
print("Percent of Volume Labelled", sum([d["Labelled Volume (nm^3)"] for d in ds])/sum([d["Used Volume (nm^3)"] for d in ds]))
print("Average Number of Objects", sum([d["Number of Objects"] for d in ds])/len(ds))
print("Average Skeleton Length", sum([d["Average Skeleton Length (nm)"] for d in ds])/len(ds))
print(df.to_latex().replace("%", "\%").replace("nm^3", "$nm^3$"))
df

/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/oblique.zarr.zip raw/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/apical.zarr.zip raw/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/spine.zarr.zip raw/s1
Average Dataset Volume 115779525066.66667
Average Labelled Volume 60391130133.333336
Percent of Volume Labelled 0.5216045764444075
Average Number of Objects 486.6666666666667
Average Skeleton Length 2536.0
\begin{tabular}{llrrrlrrrrrrrr}
\toprule
 & Voxel Size & Total Volume ($nm^3$) & Labelled Volume ($nm^3$) & Used Volume ($nm^3$) & Percent of Used Volume Labelled & Number of Objects & Number of Nodes in Skeleton & Number of Edges in Skeleton & Sum of Skeleton Lengths (nm) & Average Skeleton Length (nm) & Standard Deviation of Skeleton Length (nm) & Shortest Skeleton Length (nm) & Longest Skeleton Length (nm) \\
Dataset &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
oblique & (50nm, 4nm, 4nm) & 76223896000 & 35179532800 & 74874880800 & 47

,Voxel Size,Total Volume (nm^3),Labelled Volume (nm^3),Used Volume (nm^3),Percent of Used Volume Labelled,Number of Objects,Number of Nodes in Skeleton,Number of Edges in Skeleton,Sum of Skeleton Lengths (nm),Average Skeleton Length (nm),Standard Deviation of Skeleton Length (nm),Shortest Skeleton Length (nm),Longest Skeleton Length (nm)
Dataset,,,,,,,,,,,,,
oblique,"(50nm, 4nm, 4nm)",76223896000,35179532800,74874880800,47.0%,370,116037,115667,962575,2602,10220,101,118834
apical,"(50nm, 4nm, 4nm)",269791200000,139581124000,265664320000,52.5%,980,420672,419692,3310874,3378,19593,96,376760
spine,"(50nm, 4nm, 4nm)",7089062400,6412733600,6799374400,94.3%,110,17656,17546,179038,1628,2672,62,18741


In [189]:
6799374400 / 7089062400

0.9591359218392548

In [3]:
dataset = "SynapseWeb/team_dentate/BBCHZ"
z = get_dataset_path(dataset + ".zarr")
s = get_dataset_path(dataset + ".graphml")

In [4]:
skeletons = nx.read_graphml(s)

In [5]:
num_edges = skeletons.number_of_edges()
num_nodes = skeletons.number_of_nodes()

In [6]:
num_objects = nx.number_connected_components(skeletons)

In [7]:
connected_components = nx.connected_components(skeletons)
connected_components = list(connected_components)

In [8]:
cc = connected_components[0]

In [12]:
from funlib.evaluate import (
    rand_voi,
    expected_run_length,
    get_skeleton_lengths,
)

In [13]:
skeleton_lengths = get_skeleton_lengths(
  skeletons,
  skeleton_position_attributes=["position_z", "position_y", "position_x"],
  skeleton_id_attribute="id",
  store_edge_length="length"
)

In [21]:
total_length = np.sum([l for _, l in skeleton_lengths.items()])
average_length = total_length
std_length = np.std([l for _, l in skeleton_lengths.items()])
shortest_length = min([l for _, l in skeleton_lengths.items()])
longest_length = max([l for _, l in skeleton_lengths.items()])
print(total_length, average_length, std_length, shortest_length, longest_length)

412745.794523716 412745.794523716 5456.487510407069 152.16652059555054 24156.008276939392


In [109]:
voxel_size = get_voxel_size(z, "volumes/image/s1")
shape = get_shape(z, "volumes/image/s1")
size = [v*s for v, s in zip(voxel_size, shape)]
volume = size[0] * size[1] * size[2]
volume_vx = reduce(lambda x, y: x * y, shape)
print(voxel_size, shape, size, volume, volume_vx)


/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/BBCHZ.zarr volumes/image/s1
[50, 4, 4] (50, 2763, 4616) [2500, 11052, 18464] 510160320000 637700400


In [43]:
nonzero = np.count_nonzero(image["volumes"]["neuron_ids"]["s1"])
labelled_volume = nonzero * voxel_size[0] * voxel_size[1] * voxel_size[2]
percent_labelled = labelled_volume / volume
print(labelled_volume, percent_labelled * 100)

30469896000 5.972611903646289


In [47]:
unique, counts = np.unique(image["volumes"]["neuron_ids"]["s1"], return_counts=True)

In [57]:
np.count_nonzero(counts > 1000)

87

In [91]:
mask = image["volumes"]["object_mask"]["s1"]

In [94]:
mask = np.array(mask)

In [95]:
mask.shape

(50, 2212, 3783)

In [97]:
object_indices

array([[   0,   46, 1894],
       [   0,   46, 1895],
       [   0,   46, 1896],
       ...,
       [  49, 2210, 1050],
       [  49, 2210, 1051],
       [  49, 2210, 1052]])

In [98]:
object_indices = np.argwhere(mask > 0)
min_indices = object_indices.min(axis=0)
max_indices = object_indices.max(axis=0)

In [107]:
product = reduce(lambda x, y: x * y, max_indices - min_indices)

In [110]:
product / 637700400

0.6417735052385101